In [1]:
from sklearn.inspection import permutation_importance
import time 

from Script import *

df_train = pd.read_csv("Data/train.csv")
df_test  = pd.read_csv("Data/test.csv")

df_train = df_train.drop("i", axis=1)
y = df_train.pop("y")
df_train.insert(54,"y", y)
df_test = df_test.drop(["y", "i"], axis = 1)


df_train, df_test = preprocess_data(df_train, df_test, verbose=False)


X = df_train.iloc[:, 0:-1]
y = df_train.iloc[:, -1]



In [2]:
def prepare(X):
    cat_col = X.select_dtypes(include=['category']).columns
    X = X.drop(cat_col, axis = 1)
    return X

from sklearn.preprocessing import FunctionTransformer
drop_cat  = FunctionTransformer(prepare)

In [6]:

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

from sklearn.ensemble import GradientBoostingClassifier

In [12]:
from sklearn.neighbors import KNeighborsClassifier

import joblib 
rf_grid_search = joblib.load("Temp/rf_grid_search.joblib")

clf1 = rf_grid_search.best_estimator_



clf2 = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
    max_depth=1, random_state=0)

clf3 = make_pipeline(drop_cat, StandardScaler() , SVC(gamma='auto', probability=True, kernel='rbf'))
# clf3 = make_pipeline(drop_cat, StandardScaler() ,KNeighborsClassifier(n_neighbors=7))
print("Fitting Classifiers")
eclf1 = VotingClassifier(estimators=[
        ('rf', clf1), ('xgb', clf2), ('svm', clf3)], voting='soft')






print("soft Voting ")
hard_scores = cross_val_score(eclf1, X, y, cv=10)

print("aux stop")


import joblib
name_of_model = "rf_best+xgboost+svm_nocat_correct_types"

joblib.dump(hard_scores, "Temp/"+name_of_model+"_soft_scores"+".joblib")


print("soft voting", hard_scores.mean())


print("aux stop")

# joblib.dump(eclf1, "Temp/"+name_of_model+".joblib")

eclf1.fit(X, y)

preds_hard = eclf1.predict(df_test)
save_predictions(preds_hard, "Predictions/"+name_of_model+"_soft")







Fitting Classifiers
soft Voting 
aux stop
soft voting 0.6632316313823163
aux stop


In [8]:
joblib.load("Temp/rf_best+xgboost_correct_types_hard_scores.joblib").mean()

0.6829504335158251